In [87]:
import pandas as pd
from fastai.tabular.all import *
from pathlib import Path

In [88]:
data = pd.read_csv('tabular_data_100_features.csv')

In [89]:
data.head()

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_1271,x_1272,x_1273,x_1274,x_1275,x_1276,x_1277,x_1278,x_1279,y
0,135.0,124.0,180.0,116.0,238.0,62.0,109.0,31.0,155.0,149.0,...,69.0,166.0,87.0,149.0,25.0,255.0,71.0,101.0,42.0,1.0
1,135.0,103.0,255.0,187.0,137.0,156.0,113.0,30.0,117.0,113.0,...,255.0,200.0,72.0,176.0,99.0,225.0,29.0,0.0,148.0,1.0
2,0.0,133.0,193.0,8.0,69.0,255.0,57.0,141.0,117.0,110.0,...,143.0,91.0,125.0,62.0,177.0,172.0,0.0,243.0,0.0,1.0
3,43.0,63.0,160.0,15.0,77.0,159.0,4.0,91.0,206.0,146.0,...,215.0,129.0,199.0,255.0,113.0,36.0,43.0,0.0,197.0,1.0
4,119.0,78.0,166.0,95.0,175.0,91.0,63.0,61.0,227.0,89.0,...,63.0,195.0,61.0,197.0,70.0,80.0,0.0,0.0,220.0,1.0


In [90]:
path = Path('/home/vedrau/work/laugh_track')
cont_names = [col for col in list(data.columns) if 'x' in col]
dls = TabularDataLoaders.from_csv('tabular_data_100_features.csv', path=path, y_names="y", cat_names = [], cont_names = cont_names, procs = [FillMissing, Normalize], y_block = CategoryBlock)

In [91]:
dls.show_batch()

In [92]:
learn = tabular_learner(dls, metrics=accuracy)

In [93]:
learn.fit_one_cycle(5)

epoch,train_loss,valid_loss,accuracy,time
0,0.696198,0.680045,0.625000,00:00
1,0.538667,0.587475,0.800000,00:00
2,0.417132,0.513920,0.800000,00:00
3,0.329469,0.464652,0.825000,00:00
4,0.272655,0.443072,0.825000,00:00


In [94]:
learn.show_results()

## Try out inference: load an example file and pass it into the model

In [95]:
import tensorflow as tf

filename1 = '/home/vedrau/work/laugh_track/non_laughter_features_example.tfrecord'
filename2 = '/home/vedrau/work/laugh_track/laughter_features_example.tfrecord'
filenames = [filename1, filename2]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [96]:
# Copied from generate_audioset_training_data.ipynb but with labels removed
def parse_feature_data(raw_dataset):
    """
    This function takes the raw dataset and produces a table that contains the raw 128x10 features and a column that is the presence of laughter (0 or 1)

    Note: we will skip videos that aren't 10 seconds long
    """
    laugh_labels = [16, 17, 18, 19, 20, 21]

    extracted_table = []
    
    for raw_record in raw_dataset:
        example = tf.train.SequenceExample()
        example.ParseFromString(raw_record.numpy())

        if len(example.feature_lists.feature_list['audio_embedding'].feature) != 10:
            continue

        audio_features_10s = [np.frombuffer(example.feature_lists.feature_list['audio_embedding'].feature[i].bytes_list.value[0],np.uint8).astype(np.float32) for i in range(10)]
        # audio_features = np.frombuffer(example.feature_lists.feature_list['audio_embedding'].feature[0].bytes_list.value[0],np.uint8).astype(np.float32)
        # video_id = example.context.feature["video_id"].bytes_list.value[0]
        # start_time_secs = example.context.feature["start_time_seconds"].float_list.value[0]
        labels = example.context.feature["labels"].int64_list.value
        # end_time_secs = example.context.feature["end_time_seconds"].float_list.value[0]

        flat_audio_features = [j for sub in audio_features_10s for j in sub]

        extracted_data = flat_audio_features
        extracted_table.append(extracted_data)

    return extracted_table
    

In [97]:
example_features = parse_feature_data(raw_dataset)

2024-07-05 08:36:49.070490: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [98]:
np_example_features = np.array(example_features)

In [99]:
np_example_features.shape

(2, 1280)

In [100]:
example_dict = {f'x_{i}': np_example_features[:, i] for i in range(1280)}

In [101]:
test_df = pd.DataFrame.from_dict(example_dict)

In [102]:
test_df

,x_0,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,...,x_1270,x_1271,x_1272,x_1273,x_1274,x_1275,x_1276,x_1277,x_1278,x_1279
0,120.0,24.0,169.0,127.0,116.0,123.0,79.0,185.0,79.0,233.0,...,114.0,117.0,121.0,126.0,126.0,130.0,114.0,133.0,130.0,187.0
1,120.0,20.0,150.0,159.0,125.0,129.0,70.0,107.0,90.0,166.0,...,114.0,117.0,121.0,126.0,126.0,130.0,114.0,133.0,130.0,187.0


In [103]:
dl = learn.dls.test_dl(test_df)

In [104]:
preds = learn.get_preds(dl=dl)

In [105]:
preds

(tensor([[0.6767, 0.3233],
         [0.4893, 0.5107]]),
 None)

In [106]:
preds[0]

tensor([[0.6767, 0.3233],
        [0.4893, 0.5107]])

In [107]:
for row in preds[0]:
    if row[0] > row[1]:
        print("No laughter!")
    else:
        print("Laughter!")

No laughter!
Laughter!


It looks like the laugh detector is performing as intended!!!

## Save the model and try loading it again

In [110]:
learn.export(fname='model_export_tabular_data_100_features.pkl')

In [112]:
from fastai.learner import load_learner
learn_inference = load_learner('model_export_tabular_data_100_features.pkl')

In [113]:
learn_inference

In [114]:
learn_inference.get_preds(dl=dl)

(tensor([[0.6767, 0.3233],
         [0.4893, 0.5107]]),
 None)